# ***Data preprocessing***


## *В данном документе описан процесс предобработки исторических данных о теннисных матчах, полученных из [БД энтузиастов](https://github.com/JeffSackmann/tennis_atp "Jeff Sackmann Github repository")*


## Skills changelog:
*это первая попытка использования Machine Learning как инструмента в принципе, и поэтому
- есть только общее понимание того, что из себя представляет Data Science в целом и Machine Learning в частности;
- есть огромное желание попробовать ML на практике;
- используются любые руководства для начинающих;
- отсутствуют специфические знания и навыки, а именно - для подготовки данных, обучения моделей и т.п.;
- все, чем приходится пока руководствоваться - логика и здравый смысл.


## Sources:
- Python 3 and Pandas docs;
- StackOverflow

***Примечание:*** изначально предобработка данных была реализована на Ruby (изучение Python находилось на самом начальном этапе, а желание приступить к проекту было уже давно). Однако, для удобства использования единого формата, в данной статье реализованы все те же шаги, только с использованием Python.



# Разделы:
## [1. Отбор турниров для анализа](#section1)
## [2. Очистка данных](#section2)
## [3. Непосредственная предпобработка данных](#section3)

Вначале импортируем необходимые библиотеки и, для удобства в дальнейшем, зафиксируем корневой каталог репозитория:

In [ ]:
### !pip install pandas
import pandas as pd
import numpy as np
#import urllib
import urllib.request
import os
import glob
import csv
import re
from datetime import datetime
from IPython.display import display

# Pandas display options tweaking
pd.options.display.max_columns = None
INIT_MAX_ROWS = 50
pd.options.display.max_rows = INIT_MAX_ROWS

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../../'))
print('Root directory:', ROOT_DIR)
# Downloaded files path
DOWNL_F_P = os.path.join(ROOT_DIR, 'match_data', 'match_data_downloaded')
# Preprocessed files path
PREP_F_P = os.path.join(ROOT_DIR, 'match_data', 'match_data_preprocessed')

## <a id='section1'></a>
## 1. Отбор турниров для анализа
### Для данного учебного проекта выбраны матчи Ассоциации теннисистов-профессионалов (ATP) (без учета квалификационных матчей - существует распространенное мнение, что в таких матчах игроки не демонстрируют весь свой потенциал, и, соответственно, данные об этих матчах могут носить дезинформирующий характер для ML-модели).

Так выглядит [репозиторий](https://github.com/JeffSackmann/tennis_atp "Jeff Sackmann Github repository"), содержащий информацию о матчах разных годов и не только:
![Содержание репозитория с информацией о теннисных матчах 1](../../img/notebooks/series_1/preprocessing/tennis-repo-overview-1.jpg "Tennis repo overview")
...
![Содержание репозитория с информацией о теннисных матчах 1](../../img/notebooks/series_1/preprocessing/tennis-repo-overview-2.jpg "Tennis repo overview")

Таблицы содержат строки с информацией матча:
- турнир (id турнира, название, покрытие, дата и др.);
- игроки (id, имя, рост, возраст и др.);
- очки (эйсы, двойные ошибки, подача с первой подачи, со второй, отыгранные брейк-поинты и др.).

Так выглядит таблица с данными о матчах:

In [ ]:
url = 'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2007.csv'

def show_csv(file):
    data = pd.read_csv(file, header=0, encoding="utf-8-sig", engine='python')    
    display(data)
    return data

show_csv(url)

Если посмотреть таблицы за разные года, то можно увидеть, что данные по очкам за матч (крайние правые столбцы) начинают стабильно появляться в файлах с 1991 года по текущее время. Отчеты за эти года и будут использованы в данном проекте.

Загрузим эти файлы в наш проект:

In [ ]:
def download_raw_data_files():
    cnt = 0
    for year in range(1991, 2019):
        output_template = os.path.join(DOWNL_F_P, 'atp_matches_')
        extension = '.csv'
        output = str(output_template + str(year) + extension)
        url = 'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_' + year + '.csv'
        print('Downloading', url)
        urllib.request.urlretrieve(url, output)
        cnt += 1
    print(cnt, 'files downloaded.')

download_raw_data_files()

Объединим все файлы в один CSV-файл:

In [ ]:
def _get_headers(path, file):
    file_dir = os.path.join(path, file)
    headers = ''
    with open(file_dir, 'r', encoding='utf-8-sig') as file:
        reader = csv.reader(file)
        headers = next(reader)
    return headers

def _write_headers(file, flag, headers):
    csv_merge = open(file, flag)
    csv_merge.write(",".join(headers))
    csv_merge.write('\n')
    csv_merge.close()
    
def _write_data(input_file, output_file, flag):
    with open(input_file, 'r') as input, open(output_file, flag) as output:
        reader = csv.reader(input)
        writer = csv.writer(output, quotechar=None)
        # skip first line with headers
        next(reader)
        for r in reader:
            writer.writerow(r)
            
def get_files_in_directory(path, ext):
    all_filenames = []
    for root,dirs,files in os.walk(path):
        # create a list of all CSVs with full path (i.e. directory)
        func = lambda f: os.path.join(root, f)
        all_filenames = sorted([func(file) for file in files if re.search(r'atp_matches_....\.csv$', file)])
    return all_filenames

def merge_csv_files(path, input_files_list, output_name):
    full_output = os.path.join(path, output_name)
    headers = _get_headers(path, input_files_list[0])
    
    _write_headers(full_output, 'w', headers)
    for file in input_files_list:
        _write_data(file, full_output, 'a')

        
raw_match_data_files = get_files_in_directory(DOWNL_F_P, 'csv')
merge_csv_files(DOWNL_F_P,
                raw_match_data_files,
                'atp_matches_1991-2018.csv')

Узнаем, сначала не загружая весь файл в DataFrame, сколько итоговый файл содержит строк и столбцов и сколько занимает памяти:

In [ ]:
raw_match_data_file = 'atp_matches_1991-2018.csv'

raw_data_full_path = os.path.join(DOWNL_F_P, raw_match_data_file)


with open(raw_data_full_path, "r") as file:
    reader = csv.reader(file)
    # read first line to count features
    feature_count = len(next(reader))
    # as we have already read first line (describing columns) there only left lines with match data
    row_count = sum(1 for row in reader) - 1

raw_data_file_size = os.path.getsize(raw_data_full_path) / 1024 / 1024
print('File size:', raw_data_file_size, 'MB')
print('Number of lines:', row_count)
print('Number of features:', feature_count)

Как видно, файл не очень велик, поэтому можем работать с ним в DataFrame: 

In [ ]:
match_data = pd.read_csv(raw_data_full_path, header=0, encoding="utf-8", engine='python')
match_data

Отсортируем данные так, чтобы матчи шли от старых вначале к новым в конце.
Сделать это можно упорядочив данные по:
- дате турнира;
- id турнира;
- стадии турнира (от первых матчей в турнирной сетке до финала).

Если с первыми двумя все понятно, то по третьему пункту нужно посмотреть, какие стадии вообще есть в таблице:

In [ ]:
match_data['round'].unique()

In [ ]:
Видно, что в каких-то записях отсутствует информация о стадии турнира. Посмотрим, что с этими матчами не так:

In [ ]:
match_data.loc[match_data['round'].isnull()]

Как видим, в этих строках отсутствует какая-либо информация - можем их удалить из таблицы:

In [ ]:
# use drop=True to avoid old index being added as a column
match_data = match_data.dropna(how='all').reset_index(drop=True)
match_data

Теперь убедимся, что со стадиями турнира все в порядке:

In [ ]:
# match_data_clean['round'].unique()
match_data['round'].unique()

Проверим оставшиеся записи по колонкам "дата турнира" и "id турнира" на отсутствие нулевых записей:

In [ ]:
match_data.loc[match_data['tourney_date'].isnull()]

In [ ]:
match_data.loc[match_data['tourney_id'].isnull()]

Удалим записи о матчах групповых этапов турниров (RR) (вновь используя опцию drop=True):

In [ ]:
match_data_clean = match_data[match_data['round'] != 'RR'].reset_index(drop=True)
match_data_clean

Теперь необходимо отсортировать стадии турнира в соответствии с реальной последовательностью для дальнейшей
сортировки всего DataFrame:

In [ ]:
# List of tourney rounds sequence (BR - match for the third place)
tourney_rounds = ['R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'F', 'BR']

Отсортируем весь DataFrame в соответствии с выбранными критериями:

In [ ]:
# Make a 'round' column categorical
match_data_clean['round'] = pd.Categorical(match_data_clean['round'], tourney_rounds)

# Sort data by chosen features
match_data_clean.sort_values(['tourney_date', 'tourney_id', 'round'],
                             ascending=[True, True, True],
                             inplace = True,
                             na_position='first')
match_data_clean = match_data_clean.reset_index(drop=True)
match_data_clean

Сделаем небольшую перестановку в колонках для удобства проверки сортировки.

Для этого сначала получим текущие колонки:

In [ ]:
cols = match_data_clean.columns.tolist()
print(cols)

Так будет выглядеть новый порядок колонок в таблице:

In [ ]:
new_cols_order = ['tourney_date', 'tourney_id', 'tourney_name', 
                  'round', 'surface', 'draw_size', 'tourney_level', 
                  'match_num', 'winner_id', 'winner_seed', 
                  'winner_entry', 'winner_name', 'winner_hand', 
                  'winner_ht', 'winner_ioc', 'winner_age', 
                  'winner_rank', 'winner_rank_points', 'loser_id', 
                  'loser_seed', 'loser_entry', 'loser_name', 
                  'loser_hand', 'loser_ht', 'loser_ioc', 
                  'loser_age', 'loser_rank', 'loser_rank_points', 
                  'score', 'best_of', 'minutes', 
                  'w_ace', 'w_df', 'w_svpt', 
                  'w_1stIn', 'w_1stWon', 'w_2ndWon', 
                  'w_SvGms', 'w_bpSaved', 'w_bpFaced', 
                  'l_ace', 'l_df', 'l_svpt', 
                  'l_1stIn', 'l_1stWon', 'l_2ndWon', 
                  'l_SvGms', 'l_bpSaved', 'l_bpFaced']

In [ ]:
match_data_clean = match_data_clean.reindex(columns=new_cols_order)
pd.options.display.max_rows = 200
match_data_clean

Вернем настройки отображения к исходным:

In [ ]:
pd.options.display.max_rows = INIT_MAX_ROWS

Сохраним текущее состояние таблицы в CSV:

In [ ]:
clean_data_file = 'atp_matches_1991-2018_cleaned_reordered.csv'
clean_data_full_path = os.path.join(DOWNL_F_P, clean_data_file)
match_data_clean.to_csv(clean_data_full_path, encoding='utf-8-sig', index=False)

Теперь приступим к подготовке DataFrame с данными для обучения модели.



In [ ]:
print(match_data_clean['round'].unique())
print(match_data_clean['surface'].unique())
print(match_data_clean['best_of'].unique())
print(match_data_clean['draw_size'].unique())


Приведем названия колонок к единому формату:

In [ ]:
old_cols = match_data_clean.columns.tolist()
new_cols = []
for col in old_cols:
    if 'winner' in col:
        col = col.replace('winner', 'w')
    if 'loser' in col:
        col = col.replace('loser', 'l')
    new_cols.append(col)
print(new_cols)

match_data_clean.columns = new_cols
match_data_clean

Сохраним текущее состояние таблицы в CSV:

In [ ]:
clean_data_file = 'atp_matches_1991-2018_clean_final.csv'
clean_data_full_path = os.path.join(DOWNL_F_P, clean_data_file)
match_data_clean.to_csv(clean_data_full_path, encoding='utf-8-sig', index=False)

Загрузим итоговый полученный CSV в DataFrame:

In [ ]:
clean_final_data_file = 'atp_matches_1991-2018_clean_final.csv'
clean_data_full_path = os.path.join(DOWNL_F_P, clean_final_data_file)
clean_data = pd.read_csv(clean_data_full_path, header=0, encoding="utf-8-sig", engine='python')
clean_data.iloc[::-1].reset_index(drop=True)

In [ ]:
print(clean_data.columns.tolist())

### Основная идея анализа:
### Прогноз будет строиться на основе данных о предыдущих матчах игроков.

Выберем параметры, которые явно зависят от игроков (это первая проба выдвижения гипотезы - впоследствии этот список, безусловно, будет корректироваться):


Для более удобного анализа нам понадобиться история матчей в порядке от последних к ранним. 

In [ ]:
cols_to_parse = ['w_ht', 'l_ht', 'w_age', 'l_age', 'w_rank', 'l_rank', 'w_rank_points', 'l_rank_points', 'w_ace', 'l_ace', 'w_df', 'l_df', 'w_svpt', 'l_svpt', 'w_1stIn', 'l_1stIn', 'w_1stWon', 'l_1stWon', 'w_2ndWon', 'l_2ndWon']
# assume result columns as reciprocal of winner and loser parameters difference
# max, min and avg - are maximum, minimum and average values respectively on the period or matches number chosen
res_cols = ['ht', 'age', 'rank', 'rank_points', 'ace_last', 'ace_max', 'ace_mean', 'ace_min', 'df_last', 'df_max', 'df_mean', 'df_min', 'perc_1stIn_last', 'perc_1stIn_max', 'perc_1stIn_mean', 'perc_1stIn_min', 'perc_w_1stIn_last', 'perc_w_1stIn_max', 'perc_w_1stIn_mean', 'perc_w_1stIn_min', 'perc_w_2ndIn_last', 'perc_w_2ndIn_max', 'perc_w_2ndIn_mean', 'perc_w_2ndIn_min', 'winner_flag']

def _get_date(value, format='%Y%m%d'):
    date = datetime.strptime(str(int(value)), format)
    return date

def _get_df_cell_value(df, row_index, col):
    return df.iloc[[row_index]][col]

def _percent(part, whole):
    if float(whole) == 0 : return 0
    perc = float(part) * 100 / float(whole)
    return round(perc, 4)

def _get_values_list(feature, df, prefixes):
    values = []
    for i, row in df.iterrows():
        column = prefixes[i] + feature
        values.append(row[column])
    return values

def _last(feature, df, prefixes):
    column = prefixes[0] + feature
    res = round( int( df.iloc[0].loc[column] ), 4 )
    return res

def _min(feature, df, prefixes):
    res = round( int( min( _get_values_list( feature, df, prefixes ) ) ), 4 )
    return res

def _max(feature, df, prefixes):
    res = round( int( max( _get_values_list( feature, df, prefixes ) ) ), 4 )
    return res

def _mean(values_list):
    res = round( sum( values_list ) / float( len( values_list ) ), 4 )
    return res

def _check_features(row, features):
    for f in features:
        if np.isnan(row[f]):
            return False
    else:
        return True

def _check_for_valid_matches(df, init_index, player_name, **params):
    valid_matches_indices = []
    init_date = _get_date(_get_df_cell_value(df, init_index, 'tourney_date'))
    row_index = init_index + 1
    
    while row_index < df.shape[0]:
        row = df.iloc[row_index]        
        valid_date = (init_date - _get_date(row['tourney_date'])).days <= params.get('period')
        valid_count = len(valid_matches_indices) <= params.get('num_max')
        valid_name = (player_name == row['w_name']) | (player_name == row['l_name'])
        
        if valid_date & valid_count:
            if valid_name:
                if _check_features(row, params.get('cols_to_parse')):
                    valid_matches_indices.append(row_index)
                else:
                    break
            row_index += 1
        else:
            break
    
    if len(valid_matches_indices) < params.get('num_min'):
        valid_matches_indices = []

    return valid_matches_indices

def _get_valid_matches(df, init_index, player_name, **params):
    valid_matches_indices = _check_for_valid_matches(df, init_index, player_name, **params)
    p1ayer_valid_matches = pd.DataFrame(columns=df.columns.tolist())
    
    if valid_matches_indices:
        for i, row_index in enumerate(valid_matches_indices):
            p1ayer_valid_matches.loc[i] = df.iloc[row_index]

    p1ayer_valid_matches.reset_index(drop=True, inplace=True)
    
    return p1ayer_valid_matches

def _get_prefixes(name, df):
    prefixes = []
    for i,row in df.iterrows():
        prefix = 'w_' if name == row['w_name'] else 'l_'
        prefixes.append(prefix) 
    
    return prefixes

def _add_features(df, prefixes):
    features_to_derive = ('perc_1stIn',
                           'perc_w_1stIn',
                           'perc_w_2ndIn')
    
    for feature in features_to_derive:
        df['w_' + feature] = 0.0
        df['l_' + feature] = 0.0
    
    for i in range(df.shape[0]):
        row = df.iloc[i]
        for pr in ('w_', 'l_'):
            df.loc[i, pr + 'perc_1stIn'] = _percent(row.loc[pr + '1stIn'], row.loc[pr + 'svpt'])
            df.loc[i, pr + 'perc_w_1stIn'] = _percent(row.loc[pr + '1stWon'], row.loc[pr + '1stIn'])
            df.loc[i, pr + 'perc_w_2ndIn'] = _percent(row.loc[pr + '2ndWon'], row.loc[pr + 'svpt'] - row.loc[pr + '1stIn'])

def _preprocess_player_valid_matches(player_name, df):
    res = {}
    prefixes = _get_prefixes(player_name, df)
    features_to_copy = ('ht',
                        'age',
                        'rank',
                        'rank_points')
    features_compound = ('ace',
                         'df',
                         'perc_1stIn',
                         'perc_w_1stIn',
                         'perc_w_2ndIn')
    
    # hard-coded derivative features addition
    _add_features(df, prefixes)
    
    for feature in features_to_copy:
        res[feature] = int(df.iloc[0].loc[prefixes[0] + feature])
    
    for feature in features_compound:
        res[feature + '_last'] = _last(feature, df, prefixes)
        res[feature + '_min'] = _min(feature, df, prefixes)
        res[feature + '_max'] = _max(feature, df, prefixes)
        res[feature + '_mean'] = _mean(_get_values_list(feature, df, prefixes))
        
    return res

def _get_res(player_one_data, player_two_data, flag):
    res = {}

    for feature in player_one_data.keys():
        diff = int(player_one_data[feature]) - int(player_two_data[feature])
        if diff == 0:
            res[feature] = 0
        else:
            res[feature] = round(1 / float(diff), 4)
    
    res['winner_flag'] = int(flag)
    
    return res

def _preprocess_result_row(df_list, index):
    res_row = {}
    
    if index % 2 == 0:
        res_row = _get_res(df_list[0], df_list[1], 1)
    else:
        res_row = _get_res(df_list[1], df_list[0], -1)
    return res_row

def _get_res_row_values(res_row_dict, cols):
    res_list = []

    for i,col in enumerate(cols):
        res_list.insert(i, res_row_dict[col])
    return res_list

def preprocess(df, **params):
    print('Preprocessing match data...')
    res_frame = pd.DataFrame(columns=res_cols)
    rows_written = 0
    res_curr_index = 0
    
    for init_index, init_row in df.iterrows():
        if init_index % 100 == 0:
            print('Preprocessing row:    ', init_index, '   Rows written:    ', res_curr_index)

        valid_matches = []
        players = [init_row['w_name'], init_row['l_name']]
        
        for player_name in players:
            valid_matches.append( _get_valid_matches( df, init_index, player_name, **params ) )
        
        # if there is a lack of information on any of features being analysed 
        if valid_matches[0].empty | valid_matches[1].empty:
            continue

        preprocessed_player_data = []

        for i,player_name in enumerate(players):
            prep_valid_matches = _preprocess_player_valid_matches(player_name, valid_matches[i])
            preprocessed_player_data.insert(i, prep_valid_matches)
        
        res_row_dict = _preprocess_result_row(preprocessed_player_data, init_index)

        res_row = _get_res_row_values(res_row_dict, res_cols)
        res_frame.loc[res_curr_index] = res_row
        res_curr_index += 1
        
        if res_curr_index % params.get('rows_to_write') == 0:
            preprocessed_data_file = 'atp_matches_1991-2018_preprocessed.csv'
            preprocessed_data_full_path = os.path.join(preprocessed_files_path, preprocessed_data_file)
            res_frame.to_csv(preprocessed_data_full_path, encoding='utf-8-sig', index=False)
    
    res_frame.winner_flag = res_frame.winner_flag.astype(int)
    return res_frame

# Preprocessing parameters are:
# - period in days during which previous matches are analysed
# - minimum number of matches to analyse
# - maximum number of matches to analyse
preprocess_params = {'rows_to_write':100,
                     'period':14,
                     'num_min':3,
                     'num_max':10,
                     'cols_to_parse':cols_to_parse,
                     'res_cols': res_cols}
reverse_clean_data = clean_data.iloc[::-1].reset_index(drop=True)

%time preprocessed_data = preprocess(reverse_clean_data, **preprocess_params)
preprocessed_data

Запишем результат в отдельный CSV-файл

In [ ]:
preprocessed_data_file = 'atp_matches_1991-2018_preprocessed_final.csv'
preprocessed_data_full_path = os.path.join(PREP_F_P, preprocessed_data_file)
preprocessed_data.to_csv(preprocessed_data_full_path, encoding='utf-8-sig', index=False)